In [46]:
import os , sys
# ensure parent folder is in Python path
sys.path.append("/home/ujjwal/learning/fastapi_learn")

# change to parent folder of 'basic'
os.chdir("/home/ujjwal/learning/fastapi_learn")

!pwd


/home/ujjwal/learning/fastapi_learn


In [47]:
%%writefile basic/curd.py
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from typing import Optional

# define model Item
class Item(BaseModel):
    name: str
    quantity: Optional[int] = 0

app = FastAPI()

items = {}


@app.post("/items")
def create(item: Item):
    name = item.name
    if name in items:
        raise HTTPException(status_code=409, detail="Item exists")
    items[name] = item
    return {"message": f"Added {name} to items."}
  
@app.get("/items")
def read(name: str):
    if name not in items:
        raise HTTPException(status_code=404, detail="Item not found")
    return items[name]  
  
@app.put("/items")
def update(item: Item):
    name = item.name
    if name not in items:
        raise HTTPException(status_code=404, detail="Item not found")
    items[name] = item
    return {"message": f"Updated {name}."}
  
@app.delete("/items")
def delete(item: Item):
    name = item.name
    if name not in items:
        raise HTTPException(status_code=404, detail="Item not found")
    del items[name]
    return {"message": f"Deleted {name}."}

Overwriting basic/curd.py


In [ ]:
import subprocess

os.system("fuser -k 8001/tcp")  # kills anything using port 8001


# run FastAPI in background
proc = subprocess.Popen(["uvicorn", "basic.curd:app", "--reload", "--port=8001"])


INFO:     Will watch for changes in these directories: ['/home/ujjwal/learning/fastapi_learn']
INFO:     Uvicorn running on http://127.0.0.1:8001 (Press CTRL+C to quit)
INFO:     Started reloader process [285828] using WatchFiles
INFO:     Started server process [285830]
INFO:     Waiting for application startup.
INFO:     Application startup complete.


In [49]:
!curl  "http://127.0.0.1:8001"

INFO:     127.0.0.1:50766 - "GET / HTTP/1.1" 404 Not Found
{"detail":"Not Found"}

In [50]:
!curl  "http://127.0.0.1:8001/items" |jq .


INFO:     127.0.0.1:50774 - "GET /items HTTP/1.1" 422 Unprocessable Entity
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    90  100    90    0     0  83643      0 --:--:-- --:--:-- --:--:-- 90000
{
  "detail": [
    {
      "type": "missing",
      "loc": [
        "query",
        "name"
      ],
      "msg": "Field required",
      "input": null
    }
  ]
}


# Post 

In [51]:
!curl -X POST \
  -H 'Content-Type: application/json' \
  -d '{"name": "rock"}' \
  http://localhost:8001/items


INFO:     127.0.0.1:50776 - "POST /items HTTP/1.1" 200 OK
{"message":"Added rock to items."}

## get

In [52]:
!curl "http://localhost:8001/items?name=rock"

INFO:     127.0.0.1:50788 - "GET /items?name=rock HTTP/1.1" 200 OK
{"name":"rock","quantity":0}

## update the quantity

In [53]:
!curl -X PUT \
  -H 'Content-Type: application/json' \
  -d '{"name": "rock", "quantity": 100}' \
  http://localhost:8001/items

INFO:     127.0.0.1:50796 - "PUT /items HTTP/1.1" 200 OK
{"message":"Updated rock."}

In [54]:
#get to see the update
!curl "http://localhost:8001/items?name=rock"

INFO:     127.0.0.1:50804 - "GET /items?name=rock HTTP/1.1" 200 OK
{"name":"rock","quantity":100}

## delete 

In [55]:
!curl -X DELETE \
  -H 'Content-Type: application/json' \
  -d '{"name": "rock"}' \
  http://localhost:8001/items


INFO:     127.0.0.1:50820 - "DELETE /items HTTP/1.1" 200 OK
{"message":"Deleted rock."}

In [56]:
#get
!curl "http://localhost:8001/items?name=rock"

INFO:     127.0.0.1:50836 - "GET /items?name=rock HTTP/1.1" 404 Not Found
{"detail":"Item not found"}

In [57]:
os.system("fuser -k 8001/tcp")  ## stop 

 285828 285830

8001/tcp:           


0